# **The Simple NN Model**

In [123]:
import random

class SimpleNN:
    def __init__(self, sizeofInput, sizeofHidden, sizeofOutput):
        self.sizeofHidden = sizeofHidden
        self.sizeofInput = sizeofInput
        self.sizeofOutput = sizeofOutput

        self.weights_input_hidden = [[random.random() for _ in range(sizeofHidden)] for _ in range(sizeofInput)]
        self.bias_hidden = [random.random() for _ in range(sizeofHidden)]

        self.weightsHiddenOutput = [random.random() for _ in range(sizeofHidden)]
        self.bias_output = random.random()

    """relu activation function"""
    def relu(self, x):
        return max(0, x)

    """derivative of the relu function."""
    def reluDerivative(self, x):
        return 1 if x > 0 else 0

    """this the func. that will do the forward pass in the network.
    it takes the list of inupt values and retrun output value and hidden layer output"""
    def ForwardPropagation(self, inputs):
        hiddenlayerActivation = []
        for j in range(self.sizeofHidden):
            activation = sum(inputs[i] * self.weights_input_hidden[i][j] for i in range(self.sizeofInput))
            activation += self.bias_hidden[j]
            hiddenlayerActivation.append(activation)

        hiddenlayerOutput = [self.relu(x) for x in hiddenlayerActivation]

        output = sum(hiddenlayerOutput[j] * self.weightsHiddenOutput[j] for j in range(self.sizeofHidden))
        output += self.bias_output

        return output, hiddenlayerOutput

    def BackwardPropagation(self, input, hiddenLayerOutput, predicted_output, actualOutput, learningrate):
        """ this func. will do the BacwardPropagation pass to adjust weights and biases."""
        outputError = predicted_output - actualOutput
        d_output = outputError

        d_hidden_layer = [d_output * self.weightsHiddenOutput[j] * self.reluDerivative(hiddenLayerOutput[j]) for j in range(self.sizeofHidden)]

        self.weightsHiddenOutput = [self.weightsHiddenOutput[j] - learningrate * d_output * hiddenLayerOutput[j] for j in range(self.sizeofHidden)]
        self.bias_output -= learningrate * d_output

        for i in range(self.sizeofInput):
            for j in range(self.sizeofHidden):
                self.weights_input_hidden[i][j] -= learningrate * d_hidden_layer[j] * input[i]
        for j in range(self.sizeofHidden):
            self.bias_hidden[j] -= learningrate * d_hidden_layer[j]

    def train(self, trainData, epochs, learningrate):
        """ trains the neural network over a specified number of epochs."""
        for epoch in range(epochs):
            total_loss = 0
            for row in trainData:
                input = list(map(float, row[:-1]))
                actualOutput = float(row[-1])
                predicted_output, hiddenLayerOutput = self.ForwardPropagation(input)
                total_loss += (predicted_output - actualOutput) ** 2
                self.BackwardPropagation(input, hiddenLayerOutput, predicted_output, actualOutput, learningrate)
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(trainData)}')

    def predict(self, inputs):
        """predicts the output for a given input."""
        if isinstance(inputs[0], list):
            return [self.ForwardPropagation(row)[0] for row in inputs]
        else:
            return self.ForwardPropagation(inputs)[0]


# **Training on the Splited DataSet for Training**

In [124]:
def normalizeData(dataset):
    """normalize the dataset to the range [0, 1]."""
    inputs = [row[:-1] for row in dataset]
    minVals = [min(column) for column in zip(*inputs)]
    maxVals = [max(column) for column in zip(*inputs)]

    normalized_dataset = []
    for row in dataset:
        normalized_input = [(x - minVals[i]) / (maxVals[i] - minVals[i]) for i, x in enumerate(row[:-1])]
        normalized_dataset.append(normalized_input + [row[-1]])

    return normalized_dataset, minVals, maxVals

def denormalize(value, min_val, max_val):
    """denormalize the value from [0, 1] to original range."""
    return value * (max_val - min_val) + min_val

def splitdataset(dataset, train_ratio=0.8):
    """this splits the dataset into training and test sets"""
    dataset = list(dataset)  # Ensure dataset is a list
    random.shuffle(dataset)
    split_index = int(len(dataset) * train_ratio)
    train_data = dataset[:split_index]
    test_data = dataset[split_index:]
    return train_data, test_data

def load_data(filepath):
    import csv
    data = []
    with open(filepath, 'r') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)  # Skip the header row
        for row in reader:
            data.append([float(value) for value in row])
    return data


#loading the dataset and split to train and test
dataset = load_data('./winequality-red.csv')
normalized_dataset, minVals, maxVals = normalizeData(dataset)
print(normalized_dataset)

train_data, test_data = splitdataset(normalized_dataset, train_ratio=0.8)

#training with the data set
nn = SimpleNN(sizeofInput=2, sizeofHidden=3, sizeofOutput=1)
nn.train(train_data, epochs=100, learningrate=0.01)


[[0.24778761061946908, 0.3972602739726027, 0.0, 0.0684931506849315, 0.10684474123539232, 0.14084507042253522, 0.0989399293286219, 0.5675477239353917, 0.6062992125984251, 0.13772455089820362, 0.15384615384615385, 5.0], [0.2831858407079646, 0.5205479452054794, 0.0, 0.11643835616438358, 0.14357262103505844, 0.3380281690140845, 0.21554770318021202, 0.49412628487518584, 0.3622047244094489, 0.20958083832335334, 0.21538461538461545, 5.0], [0.2831858407079646, 0.4383561643835617, 0.04, 0.0958904109589041, 0.13355592654424042, 0.19718309859154928, 0.1696113074204947, 0.5088105726872254, 0.4094488188976376, 0.19161676646706588, 0.21538461538461545, 5.0], [0.5840707964601769, 0.10958904109589043, 0.56, 0.0684931506849315, 0.10517529215358933, 0.22535211267605634, 0.19081272084805653, 0.5822320117474312, 0.3307086614173229, 0.1497005988023952, 0.21538461538461545, 6.0], [0.24778761061946908, 0.3972602739726027, 0.0, 0.0684931506849315, 0.10684474123539232, 0.14084507042253522, 0.0989399293286219, 

# **Making prediction on the Splited Data for Testing**

In [125]:
# Predict and evaluate on the test set
predictions = []
actuals = []

for testRow in test_data:
    inputData = testRow[:-1]
    actualOutput = testRow[-1]
    predictedoutput = nn.predict(inputData)
    predictions.append(predictedoutput)
    actuals.append(actualOutput)

    print(f"Input: {inputData}, Actual Output: {actualOutput}, Predicted Output: {predictedoutput}")

Input: [0.17699115044247787, 0.30136986301369867, 0.14, 0.10273972602739727, 0.08681135225375627, 0.16901408450704225, 0.0812720848056537, 0.28634361233480354, 0.5354330708661417, 0.17365269461077845, 0.5076923076923076], Actual Output: 7.0, Predicted Output: 5.673849143339304
Input: [0.23008849557522126, 0.1643835616438356, 0.46, 0.08219178082191782, 0.10350584307178631, 0.323943661971831, 0.13427561837455831, 0.38693098384728414, 0.5196850393700787, 0.31137724550898205, 0.39999999999999997], Actual Output: 7.0, Predicted Output: 6.052676327194337
Input: [0.5309734513274336, 0.1643835616438356, 0.6, 0.08904109589041098, 0.23372287145242068, 0.08450704225352113, 0.04240282685512368, 0.6262848751835579, 0.23622047244094482, 0.437125748502994, 0.15384615384615385], Actual Output: 5.0, Predicted Output: 6.100309352587598
Input: [0.25663716814159293, 0.2602739726027397, 0.36, 0.3561643835616438, 0.09849749582637729, 0.22535211267605634, 0.3392226148409894, 0.5675477239353917, 0.48031496062

# **Evaluating the prediction**  

In [122]:
def meanSquarederror(predictions, actuals):
    """Calculate Mean Squared Error."""
    return sum((pred - actual) ** 2 for pred, actual in zip(predictions, actuals)) / len(predictions)

def meanAbsoluteerror(predictions, actuals):
    """Calculate Mean Absolute Error."""
    return sum(abs(pred - actual) for pred, actual in zip(predictions, actuals)) / len(predictions)

mse = meanSquarederror(predictions, actuals)
mae = meanAbsoluteerror(predictions, actuals)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 0.5921089004903622
Mean Absolute Error: 0.6185110182599378
